In [46]:
import nltk
import openai
import pandas as pd
import numpy as np
from getpass import getpass
import re


In [47]:

openai.api_key = getpass()

In [16]:
from openai.embeddings_utils import get_embedding

In [13]:
import PyPDF2

pdf_file = open('Revised ConvertRite User Manual - New_V3.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)
pdf_text=""
for page in range(num_pages):
    pdf_page = pdf_reader.pages[page]
    pdf_text += pdf_page.extract_text()
print(pdf_text)
pdf_file.close()


 
 
 
 
 
 
 
User Manual  
 
 
 
 
 
 
 
 - User Manual  | Confidential  Page | 2 
Table of Contents  
Introduction ................................ ................................ ................................ ................................ ..............  3 
High Level Process Flow  ................................ ................................ ................................ .........................  3 
Login Information  ................................ ................................ ................................ ................................ .... 3 
Workflow ................................ ................................ ................................ ................................ ..................  4 
Reconciliation  ................................ ................................ ................................ ................................ ........  16 
Other  Screens  ................................ ................................ ...........

In [42]:
words = nltk.word_tokenize(pdf_text)

# Chunk the words into segments of 100 words each, with 50-word overlap
chunk_size = 100
overlap_size = 50
chunks = []
paragraphs = pdf_text.split(' \n \n')
for i in range(0, len(words), chunk_size - overlap_size):
    chunk = words[i:i + chunk_size]
    chunks.append(chunk)
chunks_final=[]
# Print the chunks
for i in chunks:
    print(i)
    print("\n\n")
    chunks_final.append(' '.join(i))


['User', 'Manual', '-', 'User', 'Manual', '|', 'Confidential', 'Page', '|', '2', 'Table', 'of', 'Contents', 'Introduction', '................................', '................................', '................................', '................................', '..............', '3', 'High', 'Level', 'Process', 'Flow', '................................', '................................', '................................', '.........................', '3', 'Login', 'Information', '................................', '................................', '................................', '................................', '....', '3', 'Workflow', '................................', '................................', '................................', '................................', '..................', '4', 'Reconciliation', '................................', '................................', '................................', '................................', '....

In [58]:
paragraphs = [string for string in paragraphs if string.strip()]
new_paragraphs=[]
for string in paragraphs:
    print(string)
    newstring = re.sub(r"^\s*-\s*User Manual\s*\|\s*Confidential\s*Page\s*\|\s*\d+\s*", "", string)
    new_paragraphs.append(newstring)



 
User Manual 
 - User Manual  | Confidential  Page | 2 
Table of Contents  
Introduction ................................ ................................ ................................ ................................ ..............  3 
High Level Process Flow  ................................ ................................ ................................ .........................  3 
Login Information  ................................ ................................ ................................ ................................ .... 3 
Workflow ................................ ................................ ................................ ................................ ..................  4 
Reconciliation  ................................ ................................ ................................ ................................ ........  16 
Other  Screens  ................................ ................................ ................................ .....

In [65]:
new_paragraphs
paragraphs = pd.DataFrame(new_paragraphs, columns=['para'])
paragraphs

KeyError: 20

In [33]:
chunks_final[23]

'required credentials . • Cloud URL – SaaS URL and credentials will be provided by the functional team . Step 7 : Cloud Configuration : • Click on the Hamburger button – Configuration – Cloud Configuration . • Click on New –Define all re quired info ( From FBDI ) : Object Code – Ctrl File Name – Xlsm F ile Name - Sheet Name- Interface Table . • Hence Connected to cloud . We must get the structure – through load meta data step . ( Only after role object -parent object level ) . - User Manual |'

In [67]:
paragraphs['para'][:20]

0                                        \nUser Manual 
1     Table of Contents  \nIntroduction ...............
2     Introduction  \nIt is a data conversion tool b...
3     Login Information  \nLink: https://convertrite...
4     Workflow  \nClick on hamburger menu  on top le...
5      \n➢ Project, POD, Parent object and child obj...
6     ➢ Export CSV – Export CSV button downloads all...
7     Creation of Project : \n• To create the projec...
8     • Project is related to POD – Hence select the...
9     Creation of Parent Object:  \n• Click on the h...
10    • If the project is not listed under actual va...
11    Creation of Child Object:  \n• Search for look...
12     \nStep 2: \nAssign POD  and Project : \n• Cli...
13                                                     
14    • Click on Load WBS – Project will be assigned...
15    • If the project is created already, search fo...
16     \n• In the SuperUser profile, click on the ha...
17                                              

In [70]:
paragraphs['embedding'] = paragraphs['para'][:10].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
paragraphs.head(10)

,para,embedding
0,\nUser Manual,"[-0.013231677003204823, 0.002899555256590247, ..."
1,Table of Contents \nIntroduction ...............,"[0.01696472056210041, 0.006556727457791567, -0..."
2,Introduction \nIt is a data conversion tool b...,"[-0.014972997829318047, -0.016418857499957085,..."
3,Login Information \nLink: https://convertrite...,"[0.002685806481167674, -0.0031879355665296316,..."
4,Workflow \nClick on hamburger menu on top le...,"[0.006566131487488747, 0.002486040350049734, -..."
5,"\n➢ Project, POD, Parent object and child obj...","[0.019584445282816887, -0.003975201863795519, ..."
6,➢ Export CSV – Export CSV button downloads all...,"[-0.028637871146202087, -0.024635016918182373,..."
7,Creation of Project : \n• To create the projec...,"[-0.005877157673239708, 0.010511070489883423, ..."
8,• Project is related to POD – Hence select the...,"[0.007372644264250994, -0.024069104343652725, ..."
9,Creation of Parent Object: \n• Click on the h...,"[0.007549575064331293, 0.013941866345703602, -..."


In [76]:
paragraphs['embedding'] = paragraphs['embedding'][:10].apply(eval).apply(np.array)


TypeError: eval() arg 1 must be a string, bytes or code object

In [78]:
from openai.embeddings_utils import cosine_similarity

search_query = "how to create pod"
search_query_emd = get_embedding(search_query,engine= "text-embedding-ada-002")
paragraphs["similarities"] = paragraphs['embedding'].apply(lambda x: cosine_similarity(x, search_query_emd))

paratemp = paragraphs.head(10)

In [88]:
paratemp.sort_values('similarities',ascending=False)[:2]['para']


8    • Project is related to POD – Hence select the...
5     \n➢ Project, POD, Parent object and child obj...
Name: para, dtype: object

In [89]:
paratemp['para'][5]

' \n➢ Project, POD, Parent object and child object  (object code)  are \nMaster data.  \nSTEP 1:  \nCreation  of POD:  \nPOD - The Product Oriented Delivery (POD) model is  a software development strategy that centers on \nbuilding small cross -functional teams that own specific tasks or requirements for a project.  \nA project can be created in different PODs or many Projects in one POD can be \ncreated.  \n• To create  a POD in ConvertRite, click on hamburger button - master data - \nlookup workbench - search for POD – click on add at lookup values  – add POD \nname.  (POD  is independent - doesn’t rel y on any other values) – save.'